### **Proyecto Final parte 1**



# Deteccción de metafora mediante un modelo de red convolucional utilizando Glove embeddings.
* Materia: Reconcocimiento de Patrónes
* Profesor: Dr. Ricardo Barrón
*Alumna: Ericka Deyanira Ovando

Referencias
https://ichi.pro/es/como-implementar-cnn-para-tareas-de-pnl-como-clasificacion-de-oraciones-243737024726156

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# **Preprocesamiento de la base de datos: Trofi**

Trofi DataSet: (Birke and Sarkar, 2006), que consiste en 2,146 oraciones metaforicas y 1593 oraciones literales de 51 verbos en ingles.

In [ ]:
from tqdm import tqdm
import torch
import numpy as np
import mmap
import ast
import csv
from torch.utils.data import Dataset
import torch.nn as nn
from torch.autograd import Variable

In [ ]:
def OpenData(dataSet):
    """Se genera una lista con cada oración del archivo de texto"""
    datos = []
    with open(dataSet) as fname:
        lineas = fname.readlines()
        for linea in lineas:
            datos.append(linea.strip('./.'))
    return datos

Literal =OpenData('Literal.txt')
NoLiteral =OpenData('NoLiteral.txt')

In [ ]:
def texto(TXT):

  oraciones=[]
  str1='ws'
  str2='U'
  str3='N'
  str4='L'
  i=0
  for i in range (len(TXT)):
      linea=TXT[i]
      if str1 in linea:
          if str2 in linea:
              n=linea.index('U')
              oraciones.append(linea[n+2:len(linea)-3])
          if str3 in linea:
              n=linea.index('N')
              oraciones.append(linea[n+2:len(linea)-3])
          if str4 in linea:
              n=linea.index('L')
              oraciones.append(linea[n+2:len(linea)-3])

  return oraciones

LiteralSentences= texto(Literal)
NonLiteralSentences= texto(NoLiteral)
print(len(Literal))
print(len(NoLiteral))


3699
2741


In [ ]:
import nltk
from nltk.corpus import stopwords
import string
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize, sent_tokenize

def WordsList(TXT):
  #Se eliminan los signos de puntuación de las oraciónes, se "tokenisa"  y se genera una lista de lista con los tokens
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    numbers=['1', '2', '3','4' ,'5', '6', '7', '8', '9', '0']
    stpwrds = list(string.punctuation)+numbers
    tokens=[]
    Filter=[]
    j=0
    tokenizer = RegexpTokenizer(r'\w+')

    for j in range (len(TXT)):
            sentence=TXT[j]
            tokens. append(tokenizer.tokenize(sentence))
            filtered_sentence = [w for w in tokens[-1] if not w in stpwrds]
            Filter.append(filtered_sentence)

    return Filter

In [ ]:
filtLiteral=WordsList(LiteralSentences)
filtNonLiteral=WordsList(NonLiteralSentences)

import math
def FilterSentences(oraciones, number):
  #Reescribe las oraciones sin signos de puntuación
  #Number: genera el arreglo según la clasificación [0 o 1 para este ejemplo]"""
  arreglo=[]
  lon=[]
  voc=[]
  taste=[any]
  for i in range (len(oraciones)):
    s=oraciones[i]
    a=" "
    voc=" "
    lon.append(len(s))
    if len(s)==len(s):  #25 es la longitud de las oraciones a analizar
      for j in range (len(s)):
        a=a+ " "+ s[j] #se escribe la oración

      if i%2!=0:
        arreglo.append(a)
        voc= voc+a
      else:
        taste.append(a)
        voc= voc+a


  n=sum(lon)/len(lon)
  if number==0: m=np.zeros(len(arreglo))
  if number==1: m=np.ones(len(arreglo))
  return arreglo, m, taste, voc
print(len(filtLiteral))
print(len(filtNonLiteral))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
4475
3257


In [ ]:
FilterLiteralSentences, arrayZeros, LiteralTaste, vocLit=FilterSentences(filtLiteral, 0)
FilterNonLiteralSentences, arrayOnes, NoLiteralTaste, vocNonLit= FilterSentences(filtNonLiteral, 1)

print(len(FilterLiteralSentences))

2237


**Se obtiene cuatro listas para procesar.**

1.   Sentence Train
2.   Label train
3.   Sentence Test
4.   Label Test

In [ ]:
sentencesTrain=FilterLiteralSentences[1:1610]+FilterNonLiteralSentences[1:1610]   # se etiqueta con 0= lenguaje literal, 1= lenguaje metaforico
labelsTrain= np.concatenate((arrayZeros[1:1610],arrayOnes[1:1610]),axis=0)

sentenceTest= LiteralTaste[1:101]+NoLiteralTaste[1:101]
labelsTest= np.concatenate((arrayZeros[1:101],arrayOnes[1:101]),axis=0)

Se genera una lista de entrenamiento con los conjuntos: Sentence Training y Label Training para el entrenamiento del vocabulario con GloVe.

In [ ]:
AllSentences_Training= FilterLiteralSentences[0:len(FilterLiteralSentences)]+ FilterNonLiteralSentences[0:len(FilterNonLiteralSentences)]

### **Se generan los embeddings a partir de Glove**
Se toma de referencia:
https://colab.research.google.com/github/pytorch/ignite/blob/master/examples/notebooks/TextCNN.ipynb#scrollTo=3IQd3EVbQvTo

Se cargan las instancias necesarias para el proyecto

In [ ]:
!pip install pytorch-ignite torchtext==0.9.1 spacy
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.0 MB 1.5 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


# **Se descargan las librerias necesarias**

In [ ]:
import random
from torchtext.vocab import GloVe
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from ignite.engine import Engine, Events
from ignite.metrics import Accuracy, Loss, RunningAverage
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.contrib.handlers import ProgressBar
from ignite.utils import manual_seed

SEED = 1234
manual_seed(SEED)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from torchtext.data.utils import get_tokenizer
tokenizer = get_tokenizer("spacy")

# **Se obtenen los vectores a partir del vocabulario del conjunto de entrenamiento**

In [ ]:
from collections import Counter
from torchtext.vocab import Vocab

counter = Counter()
for i in range (len(AllSentences_Training)):
  sentence=AllSentences_Training[i]
  counter.update(tokenizer(sentence))
print(counter)

vocab=Vocab(counter,  min_freq=10,  vectors=GloVe(name='6B', dim=100, cache='/tmp/glove/'))

Counter({'the': 4897, '  ': 3865, 'to': 2663, 'of': 2429, 'and': 2110, 'a': 2091, 'in': 1709, 'that': 1105, 's': 919, 'for': 807, 'is': 722, 'on': 595, 'it': 574, 'with': 546, 'by': 540, 'The': 517, 'as': 514, 'Mr': 476, 'said': 437, 'at': 430, 'was': 409, 'he': 406, 'have': 394, 'are': 393, 'from': 391, 'has': 373, 'its': 334, 't': 334, 'S': 330, 'n': 328, 'an': 327, 'be': 325, 'his': 324, 'will': 310, 'they': 265, 'or': 261, 'would': 260, 'who': 252, 'into': 249, 'one': 245, 'out': 244, 'up': 235, 'which': 233, 'but': 227, 'their': 225, 'U': 224, 'more': 219, 'year': 213, 'says': 210, 'were': 208, 'RRB': 202, 'this': 196, 'had': 190, 'million': 185, 'market': 180, 'been': 179, 'about': 175, 'when': 171, 'than': 170, 'down': 163, 'But': 161, 'over': 160, 'new': 158, 'not': 156, 'other': 153, 'all': 148, 'some': 142, 'two': 141, 'off': 140, 'I': 140, 'can': 138, 'could': 137, 'A': 134, 'last': 134, 'company': 132, 'LRB': 131, 'years': 128, 'after': 128, 'if': 127, 'people': 126, 'In': 

Los atributos de la clase `Vocab':
* `extend` is used to extend the vocabulary
* `freqs` is a dictionary of the frequency of each word
* `itos` is a list of all the words in the vocabulary.
* `stoi` is a dictionary mapping every word to an index.
* `vectors` is a torch.Tensor of the downloaded embeddings

In [ ]:
print("The length of the new vocab is", len(vocab))
new_stoi = vocab.stoi
print("The index of '<BOS>' is", new_stoi['<BOS>'])
new_itos = vocab.itos
print("The token at index 2 is", new_itos[2])

The length of the new vocab is 1347
The index of '<BOS>' is 0
The token at index 2 is the


**Ejemplo de Glove embeding:**

In [ ]:
text_transform = lambda x: [vocab[token] for token in tokenizer(x)]
label_transform = lambda x: 1 if x == 'met' else 0   #si es metafora ==1 , si es literal==0
print(FilterNonLiteralSentences[0])
print(text_transform(FilterNonLiteralSentences[0]))
# Print out the output of text_transform
print("input to the text_transform:", "here is an example")
print("output of the text_transform:", text_transform("here is an example"))

  But in the short term it will absorb a lot of top management s energy and attention says Philippe Haspeslagh a business professor at the European management school Insead in Paris
[3, 62, 8, 2, 501, 573, 14, 35, 189, 7, 408, 5, 679, 289, 10, 1062, 6, 1047, 50, 0, 0, 7, 136, 0, 21, 2, 368, 289, 0, 0, 8, 0]
input to the text_transform: here is an example
output of the text_transform: [165, 12, 32, 446]


**Se obtienen vectores de mi conjunto para entrenamiento**

In [ ]:
def processTex(texto):
  #se genera una lista de arreglos con la representación vectorial de cada oración
  Newlist=[]
  for i in range( len(texto)):
    newSentence= texto[i]
    m=(text_transform(newSentence))

    Newlist.append(np.asarray(m))
  return Newlist

Definimos train_process y test_process

In [ ]:
train_process=processTex(sentencesTrain)
test_process= processTex(sentenceTest)

In [ ]:
train_list= sentencesTrain

Corroboramos label_train y label_test

In [ ]:
label_train=labelsTrain
label_test=labelsTest

Se establece una longitud estandar para los vectores de 100, los espacios vacios se rellenan con ceros.

In [ ]:
from keras.preprocessing.sequence import pad_sequences
maxlen = 100

Xcnn_train = pad_sequences(train_process, padding='post', maxlen=maxlen)
Xcnn_test = pad_sequences(test_process, padding='post', maxlen=maxlen)
print(Xcnn_train[4, :])

[  3 764   0 506 502  50   2   0   0 214  33   0   0 123   2 364   5   0
   0  44   0   4 189   2   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


# **Modelo CNN utilizando Keras**

In [ ]:
 from keras.models import Sequential
 from keras import layers

In [ ]:
embedding_dim = 200
vocab_size= len(vocab)
textcnnmodel = Sequential()
textcnnmodel.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
textcnnmodel.add(layers.Conv1D(128, 5, activation='relu'))
textcnnmodel.add(layers.GlobalMaxPooling1D())

textcnnmodel.add(layers.Dense(10, activation='relu'))
textcnnmodel.add(layers.Dense(1, activation='sigmoid'))
textcnnmodel.compile(optimizer='adam',
               loss='binary_crossentropy',
               metrics=['accuracy'])
textcnnmodel.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 200)          269400    
                                                                 
 conv1d_2 (Conv1D)           (None, 96, 128)           128128    
                                                                 
 global_max_pooling1d_2 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 10)                1290      
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 398,829
Trainable params: 398,829
Non-trainable params: 0
________________________________________________

Se corrobora la longitud de mis conjuntos taste y training:

In [ ]:
print(len(label_test))
print(len(Xcnn_test[0]))
print(len(label_train))
print(len(Xcnn_train[0]))


200
100
3218
100


In [ ]:
vocab_size=len(vocab)
vocab_size

1347

In [ ]:
#def Arr(lista):
#  t=[]
#  for i in range(len(lista)):
#    x= lista[i]
#    print(np.asarray(x))
#  return t

In [ ]:
import tensorflow as tf

label_test = tf.stack(label_test)
label_train = tf.stack(label_train)
Xcnn_train= tf.stack(Xcnn_train)
Xcnn_test= tf.stack(Xcnn_test)

1) Se inicializa el modelo y se obtiene la evaluación del entrenamiento y del conjunto de prueba.

In [ ]:
textcnnmodel.fit(Xcnn_train, label_train,
                     epochs=10,
                     verbose=False,
                     validation_data=(Xcnn_test, label_test),
                     batch_size=10)
loss, accuracy = textcnnmodel.evaluate(Xcnn_train, label_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = textcnnmodel.evaluate(Xcnn_test, label_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9947
Testing Accuracy:  0.6750


Para clasificar todo el  conjunto de prueba se desarrolla la siguiente función que nos permite detallar los resultados y conocer

1) El texto metaforico clasificado correctamente

2) El texto metaforico no clasificado como metafora

3) El texto literal clasificado correctamente

4) El texto literal clasificado incorrectamente.

In [ ]:
def clasificar(test_, s, l):
  literal=0
  metafora=0
  Falseliteral=0
  Falsemetafora=0

  predictions= textcnnmodel.predict(test_)
  longt= len(test_)
  for i in range(len(predictions)):
    prob= predictions[i]
    clasification= l[i]
    if prob<=.51 and clasification==0:
      print(s[i], 'cLASSIFICATION: literal', 'CLASS:', l[i] ) # probabiidaad <.5 de que sea 1 es = literal
      literal= literal+1
    if prob<=.51 and clasification!=0:
      print(s[i], 'cLASSIFICATION: literal', 'CLASS:', l[i] ) # probabiidaad <.5 de que sea 1 (literal) mal clasificado
      Falseliteral= Falseliteral+1
    if prob>=.51 and clasification==1:
      print(s[i], 'CLASIFICATION: metaphore', 'CLASS:', l[i] ) #probabilidad de que sea >.5 de que sea metafora
      metafora= metafora+1
    if prob>=.51 and clasification!=1:
      print(s[i], 'CLASIFICATION: metaphore', 'CLASS:', l[i] ) #probabilidad de que sea metafora >.5
      Falsemetafora= Falsemetafora+1
  print("REULTADO FINAL:", 'Metafora', metafora)
  print("REULTADO FINAL:", 'FalseMetafora', Falsemetafora)
  print("REULTADO FINAL:", 'Literal', literal)
  print("REULTADO FINAL:", 'FalseLiteral', Falseliteral)



In [ ]:
clasificar(Xcnn_test, sentenceTest, label_test)

  The Monitor s losses are absorbed by the church s working fund which reportedly has declined during the past two years to about 200 million from more than 280 million largely because of the stock market crash CLASIFICATION: metaphore CLASS: tf.Tensor(0.0, shape=(), dtype=float64)
  In most instances executives at the absorbed companies are the merger casualties says Steven Potter of Russell Reynolds Associates CLASIFICATION: metaphore CLASS: tf.Tensor(0.0, shape=(), dtype=float64)
  Emulating Japan s broad based conglomerates Samsung Electronics Co will officially absorb Samsung Semiconductor Telecommunications Co CLASIFICATION: metaphore CLASS: tf.Tensor(0.0, shape=(), dtype=float64)
  Operations at metal and wood coffin manufacturing plants in Missouri Indiana and Pennsylvania will remain open and in fact be expanded with additional shifts to absorb some of the workers and production idled elsewhere CLASIFICATION: metaphore CLASS: tf.Tensor(0.0, shape=(), dtype=float64)
  S Departm

# **Test para el conjunto de entrenamiento de Metafora:**

In [ ]:
test_NoLiteral= processTex(NoLiteralTaste[1:len(arrayOnes)])
NoLiteral_label= arrayOnes[1:len(arrayOnes)]
#1= lenguaje metaforico o no literal
#test_NoLiteral= processTex(NoLiteralTaste[1:2])
#print(NoLiteralTaste[1:2])
#NoLiteral_label= arrayOnes[1:2]


In [ ]:
from keras.preprocessing.sequence import pad_sequences
maxlen = 100

Xcnn_train = pad_sequences(train_process, padding='post', maxlen=maxlen)
Xcnn_test_NoLiteral= pad_sequences(test_NoLiteral, padding='post', maxlen=maxlen)
print(Xcnn_train[4, :])

[  3 764   0 506 502  50   2   0   0 214  33   0   0 123   2 364   5   0
   0  44   0   4 189   2   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [ ]:
textcnnmodel.fit(Xcnn_train, label_train,
                     epochs=10,
                     verbose=False,
                     validation_data=(Xcnn_test_NoLiteral, NoLiteral_label),
                     batch_size=10)
loss, accuracy = textcnnmodel.evaluate(Xcnn_train, label_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = textcnnmodel.evaluate(Xcnn_test_NoLiteral, NoLiteral_label, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9947
Testing Accuracy:  0.7376


In [ ]:
clasificar(Xcnn_test_NoLiteral, NoLiteralTaste[1:len(arrayOnes)] , NoLiteral_label)


  That merger valued at about billion helped fuel a succession of financial marriages in which brokerage firms were absorbed by larger diversified financial services companies cLASSIFICATION: literal CLASS: 1.0
  R J Reynolds Tobacco Co has been a rich fat operation which absorbed 400 people from the old corporate headquarters staff with hardly a burp CLASIFICATION: metaphore CLASS: 1.0
  During the past 18 months the Houston based unit of Texas Air Corp absorbed several carriers in the process inheriting a hodge podge of different aircraft cLASSIFICATION: literal CLASS: 1.0
  They stressed that many of the costs absorbed this quarter such as the firing up of USX s Fairfield facility will be a benefit later on cLASSIFICATION: literal CLASS: 1.0
  The decline resulted from costs of absorbing Stevens and from a marketing miscue in Arrow shirts cLASSIFICATION: literal CLASS: 1.0
  Management of three of the five highest yielding general purpose funds as ranked by the Donoghue Organization

IndexError: ignored

# **Test para el conjunto de entrenamiento de lenguaje literal:**

In [ ]:
test_Literal= processTex(LiteralTaste[1:len(arrayZeros)])
Literal_label= arrayZeros[1:len(arrayZeros)]


In [ ]:
from keras.preprocessing.sequence import pad_sequences
maxlen = 100

Xcnn_train = pad_sequences(train_process, padding='post', maxlen=maxlen)
Xcnn_test_Literal= pad_sequences(test_Literal, padding='post', maxlen=maxlen)
print(Xcnn_train[4, :])

[  3 764   0 506 502  50   2   0   0 214  33   0   0 123   2 364   5   0
   0  44   0   4 189   2   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [ ]:
textcnnmodel.fit(Xcnn_train, label_train,
                     epochs=10,
                     verbose=False,
                     validation_data=(Xcnn_test_Literal, Literal_label),
                     batch_size=10)
loss, accuracy = textcnnmodel.evaluate(Xcnn_train, label_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = textcnnmodel.evaluate(Xcnn_test_Literal, Literal_label, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9947
Testing Accuracy:  0.5671


In [ ]:
clasificar(Xcnn_test_Literal, LiteralTaste[1:len(arrayZeros)] , Literal_label)


  The Monitor s losses are absorbed by the church s working fund which reportedly has declined during the past two years to about 200 million from more than 280 million largely because of the stock market crash CLASIFICATION: metaphore CLASS: 0.0
  In most instances executives at the absorbed companies are the merger casualties says Steven Potter of Russell Reynolds Associates cLASSIFICATION: literal CLASS: 0.0
  Emulating Japan s broad based conglomerates Samsung Electronics Co will officially absorb Samsung Semiconductor Telecommunications Co CLASIFICATION: metaphore CLASS: 0.0
  Operations at metal and wood coffin manufacturing plants in Missouri Indiana and Pennsylvania will remain open and in fact be expanded with additional shifts to absorb some of the workers and production idled elsewhere cLASSIFICATION: literal CLASS: 0.0
  S Department of Labor report prepared by the U S Embassy here said the country continues to rely heavily on export of labor tramsmigration and the capabili